<div style="text-align: center;">
  <img src="https://github.com/Hack-io-Data/Imagenes/blob/main/01-LogosHackio/logo_amarillo@4x.png?raw=true" alt="esquema" />
</div>


# Laboratorio ETL: Análisis del Sistema Energético en España

## Objetivo

Durante todos los laboratorios de esta semana realizarás un proceso completo de ETL para analizar la relación entre la demanda, el consumo y la generación eléctrica en diferentes provincias de España a lo largo de un año. Además, complementarán este análisis con datos demográficos y económicos extraídos del Instituto Nacional de Estadística (INE). El **objetivo principal** del análisis es **examinar cómo la demanda, el consumo y la generación eléctrica en diferentes provincias de España a lo largo de los años están influenciados por factores demográficos y económicos, como la población y el PIB provincial**. El análisis busca identificar patrones y correlaciones entre estas variables para comprender mejor las dinámicas energéticas regionales y su relación con el desarrollo socioeconómico en España.

Antes de realizar el análisis, vamos a definir las hipótesis con las que vamos a trabajar, las cuales definirán todo tu análisis y planteamiento de los laboratorios: 

- **Hipótesis 1: La demanda eléctrica está correlacionada con la población de la provincia.** Provincias con mayor población tienden a tener una mayor demanda eléctrica.
  
- **Hipótesis 2: El crecimiento económico (medido por el PIB) está correlacionado con el consumo eléctrico.** Las provincias con un PIB más alto o en crecimiento experimentan un mayor consumo de energía.

- **Hipótesis 3: La proporción de generación renovable está relacionada con factores económicos o geográficos.** Provincias con un mayor desarrollo económico o con condiciones geográficas favorables (como más horas de sol o viento) tienden a generar más energía renovable.


## Tareas Laboratorio Carga

En este laboratorio, tu objetivo será diseñar la estructura de una base de datos relacional, crear las tablas necesarias y cargar en ellas los datos limpios y preparados que obtuviste en el laboratorio anterior. Trabajarás con datos relacionados con la demanda, generación eléctrica, y variables socioeconómicas, almacenándolos de manera eficiente para facilitar el análisis y las consultas posteriores.


- **Diseño de la Base de Datos:** Define una estructura de base de datos relacional que sea adecuada para almacenar los datos de demanda eléctrica, generación eléctrica, población y PIB por provincia.

- **Creación de la Base de Datos:** Escribe los scripts SQL necesarios para crear las tablas definidas en la estructura de la base de datos. Asegúrate de definir correctamente los tipos de datos y las restricciones (e.g., `NOT NULL`, `UNIQUE`).

- **Carga de Datos en la Base de Datos:** Utiliza Python para cargar los datos limpios y preparados en las tablas de la base de datos. Asegúrate de que los datos se insertan correctamente, manejando posibles errores o inconsistencias en los datos.


In [1]:
import pandas as pd
import os
import sys
sys.path.append("..")
from src.sql_funcs import *
from src.dictionaries import cod_comunidades, provincias_comunidades, dict_mes, dict_year

Cargamos los datos

In [2]:
ruta_limpios = os.path.join('..\datos', 'DatosLimpios')

df_demanda = pd.read_csv(os.path.join(ruta_limpios, 'demanda.csv'), index_col = 0)
df_generacion = pd.read_csv(os.path.join(ruta_limpios, 'generacion.csv'), index_col = 0)
df_demog = pd.read_csv(os.path.join(ruta_limpios, 'demografico.csv'), index_col = 0)
df_pib = pd.read_csv(os.path.join(ruta_limpios, 'PIB.csv'), index_col = 0)
print(os.path.join(ruta_limpios, 'demografico.csv'))

..\datos\DatosLimpios\demografico.csv


Generamos el diccionario de las provincias (ya generado en notebooks anteriores).

In [3]:
df_demog_dict = pd.read_csv('..\datos\INE\datos_demograficos.csv', encoding='latin-1', sep=';')
provincias = df_demog_dict["Provincias"].unique()
import re

cods = dict()
for provincia in provincias:
    splitted = provincia.split()
    cods["".join(re.findall(pattern= r" (\D+)", string=provincia))] = splitted[0]

cod_provincias = dict()
for k,v in cods.items():
    try:
        cod_provincias[k] = int(v)
    except:
        cod_provincias[k] = v

In [4]:
tabla_prov = pd.DataFrame([provincias_comunidades]).T.reset_index().rename(mapper = {"index" : "provincia", 0 :"cod_comun"}, axis = 1)

tabla_prov["id_prov"] = tabla_prov["provincia"].map(cod_provincias)

Así se quedará nuestra estructura de las provincias clasificadas en ccaa.

In [5]:
tabla_prov.head(10)

,provincia,cod_comun,id_prov
0,Albacete,7,2
1,Alicante/Alacant,15,3
2,Almería,4,4
3,Araba/Álava,10,1
4,Asturias,11,33
5,Ávila,8,5
6,Badajoz,16,6
7,"Balears, Illes",8743,7
8,Barcelona,9,8
9,Bizkaia,10,48


In [6]:
df_demanda.sample(10)

,value,percentage,code_com,mes,year
88,16753.540,1,8744,5,2019
271,340134.022,1,6,8,2020
60,1304599.661,1,8,1,2019
130,2209449.199,1,15,11,2019
322,15792.687,1,8744,11,2020
475,890945.768,1,5,8,2021
252,756953.464,1,8742,1,2020
594,423714.488,1,16,7,2021
229,3085375.745,1,4,2,2020
227,777171.494,1,21,12,2019


In [7]:
df_generacion.sample(10)

,energia (MWh),type,code_com,mes,year,porcentaje_generacion
1165,3898.126,Otras renovables,21,5,2019,0.02976
3139,1273594.165,Eólica,17,2,2021,0.44900
3216,22969.893,Solar fotovoltaica,8743,7,2021,0.66301
2923,87206.919,Hidráulica,14,2,2021,0.21478
2143,4.727,Solar fotovoltaica,8745,11,2020,0.00902
3322,7.292,Solar fotovoltaica,8745,5,2021,0.01281
3371,18365.706,Eólica,10,7,2021,0.21863
2646,65930.749,Hidráulica,7,1,2021,0.05041
95,516315.277,Eólica,5,12,2019,0.53668
563,83446.172,Hidráulica,14,12,2019,0.27131


In [8]:
tipos_generacion = df_generacion["type"].unique()

In [9]:
crear_db('sistema_ele')

Base de datos sistema_ele creada con éxito


Vamos a separar las métricas para el PIB de los datos crudos.

In [10]:
filtro_pib = (df_pib["Ramas de actividad"] == "Valor añadido bruto total") | (df_pib["Ramas de actividad"] == "Impuestos netos sobre los productos")

In [11]:
df_calc_pib = df_pib[filtro_pib]

In [12]:
df_ramas = df_pib[~filtro_pib]

Limpiamos métricas del df de datos demográficos, ya que estos los podremos obtener cuando lo necesitemos de los datos.

In [13]:
df_demog = df_demog[~(df_demog["Españoles/Extranjeros"] == '% Extranjeros')]

In [14]:
df_demog.sample(5)

,Provincias,Edad (3 grupos de edad),Españoles/Extranjeros,Sexo,Año,Total
2146,38,65 y más,Españoles,Mujeres,2020,89531.0
1354,23,0-15 años,Españoles,Mujeres,2020,44371.0
2647,50,0-15 años,Españoles,Hombres,2020,64843.0
1659,29,65 y más,Españoles,Mujeres,2021,142131.0
2775,52,16-64 años,Españoles,Mujeres,2021,22897.0


Mapeamos las cantidades categóricas a unos índices que almacenaremos para crear las tablas más tarde.

In [15]:
# df_demog["Sexo"] = df_demog["Sexo"].map(dict_sexo)
dict_sexo = mapeo(df_demog, 'Sexo')

In [16]:
# df_demog["Españoles/Extranjeros"] = df_demog["Españoles/Extranjeros"].map(dict_nacionalidad)
dict_nacionalidad = mapeo(df_demog, "Españoles/Extranjeros")

In [17]:
df_demog.sample(5)

,Provincias,Edad (3 grupos de edad),Españoles/Extranjeros,Sexo,Año,Total
1520,27,0-15 años,2,1,2019,1046.0
22,2,16-64 años,1,2,2020,114793.0
655,11,0-15 años,2,1,2020,3636.0
2269,42,0-15 años,1,1,2020,5172.0
2312,42,65 y más,2,1,2019,82.0


In [18]:
dict_edad = mapeo(df_demog, "Edad (3 grupos de edad)")

In [19]:
dict_year = mapeo(df_demanda, "year")

In [20]:
mapeo(df_generacion, "year")

{2019: 1, 2020: 2, 2021: 3}

In [21]:
dict_type = mapeo(df_generacion, "type")

In [22]:
mapeo(df_demog, "Año")

{2021: 1, 2020: 2, 2019: 3}

In [23]:
mapeo(df_ramas, "periodo")

c:\Users\yanru\OneDrive\Documentos\Hackio\Hackio Bootcamp\Labs\Lab-ELT\notebooks\..\src\sql_funcs.py:137: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[columna] = df[columna].map(mapper)


{2021: 1, 2020: 2, 2019: 3}

In [24]:
dict_actividades = mapeo(df_ramas, "Ramas de actividad")

c:\Users\yanru\OneDrive\Documentos\Hackio\Hackio Bootcamp\Labs\Lab-ELT\notebooks\..\src\sql_funcs.py:137: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[columna] = df[columna].map(mapper)


In [25]:
mapeo(df_calc_pib, 'periodo')
dict_metricas = mapeo(df_calc_pib, "Ramas de actividad")

c:\Users\yanru\OneDrive\Documentos\Hackio\Hackio Bootcamp\Labs\Lab-ELT\notebooks\..\src\sql_funcs.py:137: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[columna] = df[columna].map(mapper)


Creamos todas las tablas necesarias para nuestra estructura de BBDD.

In [26]:
creacion_query = '''
CREATE TABLE IF NOT EXISTS comunidades (
    id_ccaa int primary key unique not null,
    nombre varchar(200) not null
);

CREATE TABLE IF NOT EXISTS provincias (
    id_provincia int primary key unique not null,
    nombre varchar(200) not null,
    id_ccaa int references comunidades(id_ccaa)
);

CREATE TABLE IF NOT EXISTS metricas (
    id_metrica int primary key unique not null,
    nombre varchar(200) not null
);

CREATE TABLE IF NOT EXISTS years (
    id_year int primary key unique not null,
    year varchar(100) 
);

CREATE TABLE IF NOT EXISTS pib_calc (
    id_pib serial primary key unique not null,
    id_provincia int references provincias(id_provincia),
    id_metrica int references metricas(id_metrica),
    id_year int references years(id_year),
    total numeric
);

CREATE TABLE IF NOT EXISTS ramas (
    id_rama int primary key unique not null,
    nombre varchar(500) not null
);

CREATE TABLE IF NOT EXISTS registro_actividad (
    id_registro_act serial primary key unique not null,
    id_provincia int references provincias(id_provincia),
    id_rama int references ramas(id_rama),
    id_year int references years(id_year),
    total numeric
);


CREATE TABLE IF NOT EXISTS meses (
    id_mes int primary key unique not null,
    nombre varchar(100) 
);

CREATE TABLE IF NOT EXISTS sexo (
    id_sexo int primary key unique not null,
    nombre varchar(100) 
);

CREATE TABLE IF NOT EXISTS edades (
    id_edad int primary key unique not null,
    intervalo varchar(100)
);

CREATE TABLE IF NOT EXISTS nacionalidades (
    id_nacionalidad int primary key unique not null,
    nombre varchar(100)
);

CREATE TABLE IF NOT EXISTS datos_demograficos (
    id_registro_demo serial primary key unique not null,
    id_provincia int references provincias(id_provincia),
    id_edad int references edades(id_edad),
    id_nacionalidad int references nacionalidades(id_nacionalidad),
    id_sexo int references sexo (id_sexo),
    id_year int references years (id_year),
    total numeric
);

CREATE TABLE IF NOT EXISTS tipo_generacion (
    id_tipo int primary key unique not null,
    nombre varchar(200)
);

CREATE TABLE IF NOT EXISTS generacion (
    id_registro_gen serial primary key unique not null,
    id_tipo int references tipo_generacion(id_tipo),
    id_ccaa int references comunidades(id_ccaa),
    id_mes int references meses(id_mes),
    id_year int references years(id_year),
    energia numeric,
    porcentaje numeric
);

CREATE TABLE IF NOT EXISTS demanda (
    id_registro_dem serial primary key unique not null,
    id_ccaa int references comunidades(id_ccaa),
    id_mes int references meses(id_mes),
    id_year int references years(id_year),
    energia numeric,
    porcentaje numeric
);

'''

Nos quedará algo como esto: 

![imagen](..\diagrama_ddbb.png)

In [27]:
conexion = establecer_conn('sistema_ele','admin', 'postgres')
query_commit(conexion, creacion_query)

Done!


Y ahora insertamos los valores correspondientes en su orden correspondiente.

In [28]:
#! Una inserción bastante mejorable --> implementación con funciones o simplemente
#! reordenación de los índices en los parámetros de 'insert into'.

insert_comun = '''
                INSERT INTO comunidades(id_ccaa, nombre)
                            VALUES(%s,%s)
                '''

lista_comunidades = [tuple([v,k]) for k,v in cod_comunidades.items()]
conexion = establecer_conn('sistema_ele','admin', 'postgres')
query_commit_many(conexion, insert_comun, lista_comunidades)

insert_prov = '''
                INSERT INTO provincias(id_provincia, nombre, id_ccaa)
                            VALUES(%s,%s,%s)
                '''

lista_provincias = [tuple([v[2],v[0],v[1]]) for v in tabla_prov.values]
conexion = establecer_conn('sistema_ele','admin', 'postgres')
query_commit_many(conexion, insert_prov, lista_provincias)

insert_metricas = '''
                INSERT INTO metricas(id_metrica, nombre)
                            VALUES(%s,%s)
                '''
lista_metricas = [tuple([v,k]) for k,v in dict_metricas.items()]
conexion = establecer_conn('sistema_ele','admin', 'postgres')
query_commit_many(conexion, insert_metricas, lista_metricas)


insert_years = '''
                INSERT INTO years(id_year, year)
                            VALUES(%s,%s)
                '''

lista_years = [tuple([int(v), int(k)]) for k, v in dict_year.items()]
conexion = establecer_conn('sistema_ele','admin', 'postgres')
query_commit_many(conexion, insert_years, lista_years)

insert_pib_calc = '''
                INSERT INTO pib_calc(id_provincia, id_metrica, id_year, total)
                            VALUES(%s,%s,%s,%s)
                '''

lista_pib_calc = [tuple(v) for v in df_calc_pib.values]
conexion = establecer_conn('sistema_ele','admin', 'postgres')
query_commit_many(conexion, insert_pib_calc, lista_pib_calc)


insert_ramas = '''
                INSERT INTO ramas(id_rama, nombre)
                            VALUES(%s,%s)
                '''

lista_actividades = [tuple([v,k]) for k,v in dict_actividades.items()]
conexion = establecer_conn('sistema_ele','admin', 'postgres')
query_commit_many(conexion, insert_ramas, lista_actividades)


insert_registro_act = '''
                INSERT INTO registro_actividad(id_provincia, id_rama, id_year, total)
                            VALUES(%s,%s,%s,%s)
                '''

lista_registro_act = [tuple(v) for v in df_ramas.values]
conexion = establecer_conn('sistema_ele','admin', 'postgres')
query_commit_many(conexion, insert_registro_act, lista_registro_act)

insert_meses = '''
                INSERT INTO meses(id_mes, nombre)
                            VALUES(%s,%s)
                '''

lista_meses = [tuple([k,v]) for k,v in dict_mes.items()]
conexion = establecer_conn('sistema_ele','admin', 'postgres')
query_commit_many(conexion, insert_meses, lista_meses)

insert_sexo = '''
                INSERT INTO sexo(id_sexo, nombre)
                            VALUES(%s,%s)
                '''

lista_sexo = [tuple([v,k]) for k,v in dict_sexo.items()]
conexion = establecer_conn('sistema_ele','admin', 'postgres')
query_commit_many(conexion, insert_sexo, lista_sexo)

insert_edades = '''
                INSERT INTO edades(id_edad, intervalo)
                            VALUES(%s,%s)
                '''

lista_edades = [tuple([v,k]) for k,v in dict_edad.items()]
conexion = establecer_conn('sistema_ele','admin', 'postgres')
query_commit_many(conexion, insert_edades, lista_edades)

insert_nacion = '''
                INSERT INTO nacionalidades(id_nacionalidad, nombre)
                            VALUES(%s,%s)
                '''

lista_nacion = [tuple([v,k]) for k,v in dict_nacionalidad.items()]
conexion = establecer_conn('sistema_ele','admin', 'postgres')
query_commit_many(conexion, insert_nacion, lista_nacion)

insert_datos_demo = '''
                INSERT INTO datos_demograficos(id_provincia, id_edad, id_nacionalidad, id_sexo, id_year, total)
                            VALUES(%s,%s,%s,%s,%s,%s)
                '''

lista_demo = [tuple(v) for v in df_demog.values]
conexion = establecer_conn('sistema_ele','admin', 'postgres')
query_commit_many(conexion, insert_datos_demo, lista_demo)

insert_tipo_gen = '''
                INSERT INTO tipo_generacion(id_tipo, nombre)
                            VALUES(%s,%s)
                '''

lista_tipo = [tuple([v,k]) for k,v in dict_type.items()]
conexion = establecer_conn('sistema_ele','admin', 'postgres')
query_commit_many(conexion, insert_tipo_gen, lista_tipo)

insert_generacion = '''
                INSERT INTO generacion(id_tipo, id_ccaa, id_mes, id_year, energia, porcentaje)
                            VALUES(%s,%s,%s,%s,%s,%s)
                '''

lista_gene = [tuple([v[1],v[2],v[3],v[4],v[0],v[-1]]) for v in df_generacion.values]
conexion = establecer_conn('sistema_ele','admin', 'postgres')
query_commit_many(conexion, insert_generacion, lista_gene)

insert_demanda = '''
                INSERT INTO demanda(id_ccaa, id_mes, id_year, energia, porcentaje)
                            VALUES(%s,%s,%s,%s,%s)
                '''

lista_demanda = [tuple([v[2],v[3],v[4],v[0],v[1]]) for v in df_demanda.values]
conexion = establecer_conn('sistema_ele','admin', 'postgres')
query_commit_many(conexion, insert_demanda, lista_demanda)

Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
